In [39]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()

In [40]:
data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

from pyspark.sql import Window
from pyspark.sql.functions import row_number, col, rank, when, sum, abs, count

In [17]:

import datetime
track_pairs = data.alias("df1").join(data.alias("df2"), "userId", "inner")\
                               .filter("df1.trackId <> df2.trackId")\
                               .select(col("df1.trackId").alias("id1"), col("df2.trackId").alias("id2"),
                                       ((col("df2.timestamp") - col("df1.timestamp"))/60).alias("timeDif"))\

In [18]:
track_pairs_weights = track_pairs.select("id1", "id2",
                                         (when(abs(col("timeDif")) < 7, 1).otherwise(0)).alias("weight"))\
                                 .groupBy("id1", "id2")\
                                 .agg(sum("weight").alias("weights"))\
                                 .filter(col("weights")>0)

In [19]:
def norm(df, key1, field, n): 
    
    window = Window.partitionBy(key1).orderBy(col(key1), col(field).desc()) 
    
    topsDF = df.select("*", row_number().over(window).alias("row_number"))\
                                    .filter(col("row_number") < n)\
                                    .drop(col("row_number"))
            
    tmpDF = topsDF.groupBy(col(key1))\
                              .agg(sum(field).alias("total_" + field))
        
    normalizedDF = topsDF.join(tmpDF, key1, "inner")\
                         .withColumn("norm_" + field, col(field)/col("total_" + field))
                

    return normalizedDF

In [20]:
normilized_weights = norm(track_pairs_weights, 'id1', "weights", 51)

In [ ]:
result = normilized_weights.orderBy(col("norm_weights").desc(), col("id1"), col("id2"))\
                            .select("id1", "id2")\
                            .take(40)

In [ ]:
#for val in result:
#    print("%s %s" % val)

798256 923706
798319 837992
798322 876562
798331 827364
798335 840741
798374 816874
798375 810685
798379 812055
798380 840113
798396 817687
798398 926302
798405 867217
798443 905923
798457 918918
798460 891840
798461 940379
798470 840814
798474 963162
798477 883244
798485 955521
798505 905671
798550 936295
798626 845438
798691 818279
798692 898823
798702 811440
798704 937570
798725 933147
798738 894170
798745 799665
798782 956938
798801 950802
798820 890393
798833 916319
798865 962662
798931 893574
798946 946408
799012 809997
799024 935246
799047 905199


In [89]:
userTrack = data.groupBy("userId", "trackId")\
                .agg(count('*').alias("weight"))

In [98]:
result2 = norm(userTrack, "userId", "weight", 1001).orderBy(col("norm_weight").desc(), 
                                                            col("userId"), 
                                                            col("trackId"))\
                                                   .select(col("userId"), col("trackId"))\
                                                   .take(40)

In [99]:
#for val in result2:
#    print("%s %s" % val)

66 965774
116 867268
128 852564
131 880170
195 946408
215 860111
235 897176
300 857973
321 915545
328 943482
333 818202
346 864911
356 961308
428 943572
431 902497
445 831381
488 841340
542 815388
617 946395
649 901672
658 937522
662 881433
698 935934
708 952432
746 879259
747 879259
776 946408
784 806468
806 866581
811 948017
837 799685
901 871513
923 879322
934 940714
957 945183
989 878364
999 967768
1006 962774
1049 849484
1057 920458


In [23]:
userArtist = data.groupBy("userId", "artistId")\
                .agg(count('*').alias("weight"))

In [25]:
result3 = norm(userArtist, "userId", "weight", 101).orderBy(col("norm_weight").desc(), 
                                                            col("userId"), 
                                                            col("artistId"))\
                                                   .select(col("userId"), col("artistId"))\
                                                   .take(40)

In [26]:
#for val in result3:
#    print("%s %s" % val)

66 993426
116 974937
128 1003021
131 983068
195 997265
215 991696
235 990642
288 1000564
300 1003362
321 986172
328 967986
333 1000416
346 982037
356 974846
374 1003167
428 993161
431 969340
445 970387
488 970525
542 969751
612 987351
617 970240
649 973851
658 973232
662 975279
698 995788
708 968848
746 972032
747 972032
776 997265
784 969853
806 995126
811 996436
837 989262
901 988199
923 977066
934 990860
957 991171
989 975339
999 968823


In [28]:
artistTrack = data.groupBy("artistId", "trackId")\
                .agg(count('*').alias("weight"))

In [30]:
result4 = norm(artistTrack, "artistId", "weight", 101).orderBy(col("norm_weight").desc(), 
                                                            col("artistId"), 
                                                            col("trackId"))\
                                                   .select(col("artistId"), col("trackId"))\
                                                   .take(40)

In [31]:
#for val in result4:
#    print("%s %s" % val)

967993 869415
967998 947428
968004 927380
968017 859321
968022 852786
968034 807671
968038 964150
968042 835935
968043 913568
968046 935077
968047 806127
968065 907906
968073 964586
968086 813446
968092 837129
968118 914441
968125 821410
968140 953008
968148 877445
968161 809793
968163 803065
968168 876119
968189 858639
968221 896937
968224 892880
968232 825536
968237 932845
968238 939177
968241 879045
968242 911250
968248 953554
968255 808494
968259 880230
968265 950148
968266 824437
968269 913243
968272 816049
968278 946743
968285 847460
968286 940006


In [ ]:
userId = 776748
userTracks = data.filter(col("userId")==userId)\
                  .join(meta, col("trackId")==col("Id") ,"inner")\
                  .select("Artist", "Name")
userArtists = data.filter(col("userId")==userId)\
                  .join(meta, col("artistId")==col("Id") ,"inner")\
                  .select("Artist", "Name")
result5 = userTracks.union(userArtists)\
                    .orderBy("Artist", "Name")\
                    .distinct().take(40)

In [ ]:
for value in result5:
    print("%s %s" % value)